<a href="https://colab.research.google.com/github/csabi0312/modelling-the-age-of-the-oldest-person-in-the-world/blob/main/NM_oldest_person60.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Def loglikelihood

In [ ]:
import numpy as np
import pandas as pd
from scipy.integrate import quad
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import copy
from tqdm import tqdm

def fGGM(x, a, b, c, gamma):
    return (np.exp(-c * x) * (1 + (a * (-1 + np.exp(b * x)) * gamma) / b) ** (-((1 + gamma) / gamma)) * (c * (b - a * gamma) + a * np.exp(b * x) * (b + c * gamma))) / b

def FGGM(x, a, b, c, gamma):
    return np.exp(-c * x) / (1 + a * gamma * (np.exp(b * x) - 1) / b) ** (1 / gamma)

listofvalues = [[1955.8138888888889, 113.17808219178082], [1955.8138888888889, 110.6986301369863], [1956.9583333333333, 111.84657534246575], [1956.9583333333333, 107.8958904109589], [1958.2472222222223, 109.17808219178082], [1958.2472222222223, 108.56438356164384], [1958.7944444444445, 109.11506849315069], [1958.7944444444445, 109.04109589041096], [1959.7666666666667, 110.01369863013699], [1959.7666666666667, 109.43561643835616], [1959.8972222222221, 109.56712328767124], [1959.8972222222221, 108.46575342465754], [1964.9972222222223, 113.56438356164384], [1964.9972222222223, 109.63561643835617], [1965.1805555555557, 109.81643835616438], [1965.1805555555557, 109.66301369863014], [1965.5972222222222, 110.08219178082192], [1965.5972222222222, 109.57808219178082], [1966.025, 110.00821917808219], [1966.025, 109.57260273972602], [1968.2222222222222, 111.76712328767124], [1968.2222222222222, 111.17534246575343], [1968.4583333333333, 111.41369863013699], [1968.4583333333333, 110.65205479452055], [1969.3416666666667, 111.53150684931506], [1969.3416666666667, 111.23835616438356], [1970.0277777777778, 111.92876712328767], [1970.0277777777778, 109.4958904109589], [1973.1555555555556, 112.62465753424658], [1973.1555555555556, 111.63561643835617], [1973.6305555555555, 112.10684931506849], [1973.6305555555555, 110.44109589041096], [1973.8333333333333, 110.64383561643835], [1973.8333333333333, 110.59726027397261], [1975.4166666666667, 112.17808219178082], [1975.4166666666667, 111.81917808219178], [1976.875, 113.28219178082192], [1976.875, 110.38082191780822], [1977.9194444444445, 111.42465753424658], [1977.9194444444445, 111.33424657534246], [1978.3166666666666, 111.72876712328767], [1978.3166666666666, 111.03013698630137], [1981.0583333333334, 113.77534246575343], [1981.0583333333334, 112.05479452054794], [1981.1861111111111, 112.17808219178082], [1981.1861111111111, 111.53698630136986], [1982.8666666666666, 113.22191780821917], [1982.8666666666666, 112.5068493150685], [1983.7833333333333, 113.42191780821918], [1983.7833333333333, 113.26301369863013], [1985.125, 114.6082191780822], [1985.125, 113.53972602739726], [1986.8055555555557, 115.21643835616439], [1986.8055555555557, 113.3945205479452], [1987.0861111111112, 113.67945205479452], [1987.0861111111112, 113.67123287671232], [1987.9888888888888, 114.56986301369864], [1987.9888888888888, 114.21369863013699], [1988.0277777777778, 114.25479452054795], [1988.0277777777778, 112.88767123287671], [1997.5916666666667, 122.44931506849315], [1997.5916666666667, 116.93150684931507], [1998.2916666666667, 117.63013698630137], [1998.2916666666667, 117.55890410958904], [1999.9972222222223, 119.26575342465753], [1999.9972222222223, 114.14246575342466], [2000.8361111111112, 114.98630136986301], [2000.8361111111112, 114.52328767123288], [2001.4305555555557, 115.11506849315069], [2001.4305555555557, 114.37260273972603], [2002.213888888889, 115.15342465753425], [2002.213888888889, 114.33698630136986], [2002.4083333333333, 114.53150684931506], [2002.4083333333333, 114.26849315068493], [2002.638888888889, 114.5013698630137], [2002.638888888889, 113.58356164383562], [2002.9944444444445, 113.93972602739726], [2002.9944444444445, 113.76986301369863], [2003.7416666666666, 114.51780821917808], [2003.7416666666666, 114.37260273972603], [2003.8666666666666, 114.4986301369863], [2003.8666666666666, 114.2027397260274], [2004.411111111111, 114.74520547945205], [2004.411111111111, 114.7068493150685], [2006.6555555555556, 116.95068493150686], [2006.6555555555556, 116.03287671232877], [2006.9444444444443, 116.32328767123288], [2006.9444444444443, 115.30684931506849], [2007.0638888888889, 115.42739726027398], [2007.0638888888889, 114.17260273972603], [2007.075, 114.18356164383562], [2007.075, 114.06575342465753], [2007.6166666666666, 114.6054794520548], [2007.6166666666666, 114.31506849315069], [2008.9027777777778, 115.6027397260274], [2008.9027777777778, 115.21095890410959], [2009.0027777777777, 115.31232876712329], [2009.0027777777777, 114.74246575342465], [2009.6944444444443, 115.43287671232876], [2009.6944444444443, 114.33972602739726], [2010.3361111111112, 114.97808219178083], [2010.3361111111112, 114.20547945205479], [2010.8416666666667, 114.71506849315068], [2010.8416666666667, 114.32328767123288], [2011.4722222222222, 114.95068493150686], [2011.4722222222222, 114.81917808219178], [2012.925, 116.27397260273973], [2012.925, 115.66849315068494], [2012.9611111111112, 115.7041095890411], [2012.9611111111112, 115.66301369863014], [2013.4472222222223, 116.14794520547945], [2013.4472222222223, 115.27123287671233], [2015.25, 117.07397260273973], [2015.25, 116.74246575342465], [2015.263888888889, 116.75616438356164], [2015.263888888889, 115.87123287671233], [2015.4611111111112, 116.06849315068493], [2015.4611111111112, 115.94794520547946], [2016.3638888888888, 116.85205479452055], [2016.3638888888888, 116.45205479452055], [2017.2888888888888, 117.37534246575342], [2017.2888888888888, 117.0986301369863], [2017.7055555555555, 117.51780821917808], [2017.7055555555555, 117.11506849315069], [2018.3055555555557, 117.71232876712328], [2018.3055555555557, 116.96986301369863]]

peaks = np.array(listofvalues)[::2]
timespeakslows = np.column_stack((peaks, np.array(listofvalues)[1::2, 1]))

def llh(t, y, z, alpha, KK, b, c, gamma, kappa, CC, birthage):
    def integrand(u):
        return np.exp(-kappa * u) * FGGM(u - birthage, KK * np.exp(-alpha * (t - u + birthage - 2000)), b, c, gamma)
    integral, _ = quad(integrand, z, y, epsabs=1.48e-09, epsrel=1.48e-09)
    return np.log(fGGM(y - birthage, KK * np.exp(-alpha * (t - y + birthage - 2000)), b, c, gamma)) + kappa * (t - z + birthage) + np.log(CC) - CC * np.exp(kappa * (t + birthage)) * integral

def loglikelihood(params):
    alpha, KK, b,gamma = params
    c=0
    CC = 3.249*10**(-11)
    birthage = 60
    kappa = 0.02085
    return -(sum([llh(timespeakslows[i, 0], timespeakslows[i, 1], timespeakslows[i, 2], alpha, KK, b, c, gamma, kappa, CC, birthage) for i in range(timespeakslows.shape[0])]) + np.log(FGGM(timespeakslows[-1, 2] - birthage, KK * np.exp(-alpha * (timespeakslows[-1, 0] - timespeakslows[-1, 2] + birthage - 2000)), b, c, gamma)))



#Optimization

In [ ]:
def optimize_parameters(alpha, KK, b, gamma ,no_of_steps=100, tol_factor=0.0001,min_tol=1e-9, max_tol=0.1, tol_treshold=0.001, tol_dec_factor=0.1, tol_inc_factor=1.1):
    bounds = [(0, 1)] * 4  # Initialize all bounds to None

    # Check if alpha is a float or a tuple with bounds
    if isinstance(alpha, tuple):
        bounds[0] = alpha
        alpha = (alpha[0] + alpha[1]) / 2  # Use the midpoint as the initial value
    else:
        bounds[0] = (alpha, alpha)  # Fixed value, set bounds to the same value

    # Repeat the same process for KK, b, and gamma
    if isinstance(KK, tuple):
        bounds[1] = KK
        KK = (KK[0] + KK[1]) / 2
    else:
        bounds[1] = (KK, KK)

    if isinstance(b, tuple):
        bounds[2] = b
        b = (b[0] + b[1]) / 2
    else:
        bounds[2] = (b, b)

    if isinstance(gamma, tuple):
        bounds[3] = gamma
        gamma = (gamma[0] + gamma[1]) / 2
    else:
        bounds[3] = (gamma, gamma)

    initial_params = [alpha, KK, b, gamma]

    likeli_scores=[]
    for i in tqdm(range(1, no_of_steps+1)):
      result = minimize(loglikelihood, initial_params, method='Nelder-Mead', bounds=bounds, tol=tol_factor)
      optimized_params = result.x
      likeli_scores.append(-loglikelihood(optimized_params))
      initial_params = copy.deepcopy(optimized_params)

      # Calculate percentage improvement
      if i > 1:
        improvement = (likeli_scores[-2] - likeli_scores[-1]) / likeli_scores[-2]
      else:
        improvement = float('inf')

      # Adjust tolerance based on improvement
      if improvement > tol_treshold:  # Example threshold for significant improvement
        tol_factor *= tol_dec_factor  # Decrease tolerance
      else:
        tol_factor *= tol_inc_factor  # Increase tolerance

      # Clamp the tolerance to the minimum value
      tol_factor = min(max(tol_factor, min_tol),max_tol)


    initial_params=list(initial_params)
    initial_params.append(likeli_scores[-1])

    return initial_params

Each parameter (alpha, K, b, gamma) can take two types of values:

* Tuple (e.g., (0,1)): in this case, the parameter value can vary within these bounds during optimization.
* Float number (e.g., 0.3): in this case, the parameter is fixed to this value during optimization.

The function returns the 4 parameter values corresponding to the best log-likelihood value and the log-likelihood itself.


In [ ]:
#Itt az alpha 0 és 0.1 között mozoghat, a KK 0.01 és 0.1 között, a b fixálva van 0.333407-re, a gamma pedig 0.395047-re
optimize_parameters(alpha=(0,0.1), KK=(0.01,0.1), b=0.333407, gamma=0.395047,no_of_steps=20)

100%|██████████| 20/20 [00:01<00:00, 10.60it/s]


[0.0, 0.01, 0.333407, 0.395047, -1193.1144859363674]

In [ ]:
optimize_parameters(alpha=(0,0.1),KK=(0.0001,0.1),b=0.137,gamma=0.149,no_of_steps=3)

100%|██████████| 3/3 [00:03<00:00,  1.11s/it]


[0.016825210634849576, 0.002112139676756749, 0.137, 0.149, -116.9537767820961]

In [ ]:
optimize_parameters(alpha=(0,0.1),KK=(0.0001,0.1),b=0.137,gamma=0.149,no_of_steps=50)

100%|██████████| 50/50 [00:22<00:00,  2.25it/s]


[0.016825210634849576, 0.002112139676756749, 0.137, 0.149, -116.9537767820961]

In [ ]:
optimize_parameters(alpha=0.0168,KK=(0.0001,0.1),b=0.137,gamma=0.149,no_of_steps=10)

100%|██████████| 10/10 [00:03<00:00,  3.07it/s]


[0.0168, 0.002115449459066359, 0.137, 0.149, -116.95401847316104]

In [ ]:
optimize_parameters(alpha=(0,.1),KK=(0.0001,0.1),b=0.137,gamma=(0,0.2),no_of_steps=10)

100%|██████████| 10/10 [00:09<00:00,  1.07it/s]


[0.016823189644043873,
 0.0021265432808935864,
 0.137,
 0.1496241565813103,
 -116.95326030911195]

In [ ]:
optimize_parameters(alpha=(0,.1),KK=(0.0001,0.1),b=(0,0.5),gamma=(0.01,0.2),no_of_steps=1)

  0%|          | 0/1 [00:00<?, ?it/s]<ipython-input-2-25d037130caf>:13: RuntimeWarning: invalid value encountered in double_scalars
  return np.exp(-c * x) / (1 + a * gamma * (np.exp(b * x) - 1) / b) ** (1 / gamma)
<ipython-input-2-25d037130caf>:23: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  integral, _ = quad(integrand, z, y, epsabs=1.48e-09, epsrel=1.48e-09)
<ipython-input-2-25d037130caf>:10: RuntimeWarning: invalid value encountered in double_scalars
  return (np.exp(-c * x) * (1 + (a * (-1 + np.exp(b * x)) * gamma) / b) ** (-((1 + gamma) / gamma)) * (c * (b - a * gamma) + a * np.exp(b * x) * (b + c * gamma))) / b
100%|██████████| 1/1 [00:07<00:00,  7.37s/it]


[0.021511866426542656,
 0.00046860861338790874,
 0.17504597218271128,
 0.19966088989778963,
 -116.8551733677509]

In [ ]:
optimize_parameters(alpha=(0,.1),KK=(0.0001,0.1),b=(0,0.5),gamma=(0.01,0.2),no_of_steps=10)

  0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-2-25d037130caf>:13: RuntimeWarning: invalid value encountered in double_scalars
  return np.exp(-c * x) / (1 + a * gamma * (np.exp(b * x) - 1) / b) ** (1 / gamma)
<ipython-input-2-25d037130caf>:23: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  integral, _ = quad(integrand, z, y, epsabs=1.48e-09, epsrel=1.48e-09)
<ipython-input-2-25d037130caf>:10: RuntimeWarning: invalid value encountered in double_scalars
  return (np.exp(-c * x) * (1 + (a * (-1 + np.exp(b * x)) * gamma) / b) ** (-((1 + gamma) / gamma)) * (c * (b - a * gamma) + a * np.exp(b * x) * (b + c * gamma))) / b
100%|██████████| 10/10 [00:14<00:00,  1.42s/it]


[0.021542226288690937,
 0.00046468025964588495,
 0.17526697272569106,
 0.2,
 -116.8547152401002]

In [ ]:
optimize_parameters(alpha=(0,1),KK=(0,1),b=0.09,gamma=(0,1),no_of_steps=10)

100%|██████████| 10/10 [00:11<00:00,  1.15s/it]


[0.011100984472805159,
 0.012085252617053757,
 0.09,
 0.08026602052545971,
 -117.26760673749304]

In [ ]:
optimize_parameters(alpha=(0,3),KK=(0,3),b=0.09,gamma=(0,3),no_of_steps=10)

  0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-2-25d037130caf>:13: RuntimeWarning: divide by zero encountered in double_scalars
  return np.exp(-c * x) / (1 + a * gamma * (np.exp(b * x) - 1) / b) ** (1 / gamma)
<ipython-input-2-25d037130caf>:10: RuntimeWarning: divide by zero encountered in double_scalars
  return (np.exp(-c * x) * (1 + (a * (-1 + np.exp(b * x)) * gamma) / b) ** (-((1 + gamma) / gamma)) * (c * (b - a * gamma) + a * np.exp(b * x) * (b + c * gamma))) / b
100%|██████████| 10/10 [00:13<00:00,  1.35s/it]


[0.011101163083950904,
 0.01208477866240596,
 0.09,
 0.08026301152135076,
 -117.26760669285133]

In [ ]:
optimize_parameters(alpha=(0,.1),KK=(0,.1),b=0.09,gamma=(0,.1),no_of_steps=10)

100%|██████████| 10/10 [00:05<00:00,  1.89it/s]


[0.011045865502069485, 0.014684065280357693, 0.09, 0.1, -118.00190893764626]

In [ ]:
optimize_parameters(alpha=(0.01,0.02),KK=(0.01,0.02),b=0.09,gamma=(0.05,0.1),no_of_steps=10)

100%|██████████| 10/10 [00:07<00:00,  1.32it/s]


[0.01110133450865787,
 0.012084660645936217,
 0.09,
 0.08026381962252223,
 -117.26760682864827]

In [ ]:
optimize_parameters(alpha=(0.005,0.05),KK=(0.005,0.03),b=0.09,gamma=(0.02,0.3),no_of_steps=10)

100%|██████████| 10/10 [00:08<00:00,  1.14it/s]


[0.011101223143704445,
 0.01208444826305248,
 0.09,
 0.08026113700483775,
 -117.26760677979718]

In [ ]:
optimize_parameters(alpha=(0.005,0.05),KK=(0.005,0.03),b=0.1,gamma=(0.02,0.3),no_of_steps=10)

100%|██████████| 10/10 [00:09<00:00,  1.10it/s]


[0.012302451336039579,
 0.008487090914166034,
 0.1,
 0.0958806712134099,
 -117.16940381679099]

In [ ]:
optimize_parameters(alpha=(0.005,0.05),KK=(0.001,0.03),b=0.11,gamma=(0.02,0.3),no_of_steps=10)

100%|██████████| 10/10 [00:07<00:00,  1.32it/s]


[0.013515753510666214,
 0.005900871057638203,
 0.11,
 0.11097774634798435,
 -117.0920220206464]

In [ ]:
optimize_parameters(alpha=(0.005,0.05),KK=(0.001,0.03),b=0.13,gamma=(0.02,0.3),no_of_steps=10)

100%|██████████| 10/10 [00:09<00:00,  1.06it/s]


[0.015962567701940995,
 0.002783649132239798,
 0.13,
 0.1398588849564705,
 -116.9816735844671]

In [ ]:
optimize_parameters(alpha=(0.005,0.5),KK=(0,1),b=0.15,gamma=(0.02,0.5),no_of_steps=10)

100%|██████████| 10/10 [00:15<00:00,  1.59s/it]


[0.018423552816753216,
 0.0012809129168726795,
 0.15,
 0.1673735327083936,
 -116.91056727840339]